In [1]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install "git+https://github.com/albumentations-team/albumentations.git"

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-u4kiogyc
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-u4kiogyc
     |████████████████████████████████| 106 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 13.6 MB/s eta 0:00:01
  Created wheel for detectron2: filename=detectron2-0.6-cp38-cp38-linux_x86_64.whl size=7338214 sha256=cd21645f934dfebeb2c8b38216ec544d6e02531400c602ec436a0ca26837f46a
  Stored in directory: /tmp/pip-ephem-wheel-cache-xi72arzr/wheels/19/ac/65/e48e5e4ec2702274d927c5a6efb75709b24014371d3bb778f2
  Created wheel for fvcore: filename=fvcore-0.1.5.post20211023-py3-none-any.whl size=60943 sha25

In [6]:
import sys
sys.path.append("utils")

In [7]:
import detectron2
import time
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer
from albumentations import *
import torch
import os
from detectron2.data import detection_utils
from utils.aug import MyMapper,MyMapper_copy_paste
from detectron2.solver.build import *
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import opendatasets as od
import warnings
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks
warnings.filterwarnings("ignore")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
#os.environ['CUDA_VISIBLE_DEVICES'] = "0,1" 

In [8]:
class Config:
  pixel_mean = [128,128,128]
  pixel_std = [13.235,13.235,13.235]
  anchor_generators_sizes = [[8], [16], [32], [64],[128]]
  anchor_generators_aspect_ratios = [[0.5, 1.0, 2.0]]

In [9]:
# 魔改mask loss为对称loss
# @torch.jit.unused
# def mask_rcnn_loss(pred_mask_logits, instances, vis_period=0):
#     """
#     Compute the mask prediction loss defined in the Mask R-CNN paper.

#     Args:
#         pred_mask_logits (Tensor): A tensor of shape (B, C, Hmask, Wmask) or (B, 1, Hmask, Wmask)
#             for class-specific or class-agnostic, where B is the total number of predicted masks
#             in all images, C is the number of foreground classes, and Hmask, Wmask are the height
#             and width of the mask predictions. The values are logits.
#         instances (list[Instances]): A list of N Instances, where N is the number of images
#             in the batch. These instances are in 1:1
#             correspondence with the pred_mask_logits. The ground-truth labels (class, box, mask,
#             ...) associated with each instance are stored in fields.
#         vis_period (int): the period (in steps) to dump visualization.

#     Returns:
#         mask_loss (Tensor): A scalar tensor containing the loss.
#     """
#     cls_agnostic_mask = pred_mask_logits.size(1) == 1
#     total_num_masks = pred_mask_logits.size(0)
#     mask_side_len = pred_mask_logits.size(2)
#     assert pred_mask_logits.size(2) == pred_mask_logits.size(3), "Mask prediction must be square!"

#     gt_classes = []
#     gt_masks = []
#     for instances_per_image in instances:
#         if len(instances_per_image) == 0:
#             continue
#         if not cls_agnostic_mask:
#             gt_classes_per_image = instances_per_image.gt_classes.to(dtype=torch.int64)
#             gt_classes.append(gt_classes_per_image)

#         gt_masks_per_image = instances_per_image.gt_masks.crop_and_resize(
#             instances_per_image.proposal_boxes.tensor, mask_side_len
#         ).to(device=pred_mask_logits.device)
#         # A tensor of shape (N, M, M), N=#instances in the image; M=mask_side_len
#         gt_masks.append(gt_masks_per_image)

#     if len(gt_masks) == 0:
#         return pred_mask_logits.sum() * 0

#     gt_masks = cat(gt_masks, dim=0)

#     if cls_agnostic_mask:
#         pred_mask_logits = pred_mask_logits[:, 0]
#     else:
#         indices = torch.arange(total_num_masks)
#         gt_classes = cat(gt_classes, dim=0)
#         pred_mask_logits = pred_mask_logits[indices, gt_classes]

#     if gt_masks.dtype == torch.bool:
#         gt_masks_bool = gt_masks
#     else:
#         # Here we allow gt_masks to be float as well (depend on the implementation of rasterize())
#         gt_masks_bool = gt_masks > 0.5
#     gt_masks = gt_masks.to(dtype=torch.float32)

#     # Log the training accuracy (using gt classes and 0.5 threshold)
#     mask_incorrect = (pred_mask_logits > 0.0) != gt_masks_bool
#     mask_accuracy = 1 - (mask_incorrect.sum().item() / max(mask_incorrect.numel(), 1.0))
#     num_positive = gt_masks_bool.sum().item()
#     false_positive = (mask_incorrect & ~gt_masks_bool).sum().item() / max(
#         gt_masks_bool.numel() - num_positive, 1.0
#     )
#     false_negative = (mask_incorrect & gt_masks_bool).sum().item() / max(num_positive, 1.0)

#     storage = get_event_storage()
#     storage.put_scalar("mask_rcnn/accuracy", mask_accuracy)
#     storage.put_scalar("mask_rcnn/false_positive", false_positive)
#     storage.put_scalar("mask_rcnn/false_negative", false_negative)
#     if vis_period > 0 and storage.iter % vis_period == 0:
#         pred_masks = pred_mask_logits.sigmoid()
#         vis_masks = torch.cat([pred_masks, gt_masks], axis=2)
#         name = "Left: mask prediction;   Right: mask GT"
#         for idx, vis_mask in enumerate(vis_masks):
#             vis_mask = torch.stack([vis_mask] * 3, axis=0)
#             storage.put_image(name + f" ({idx})", vis_mask)

#     #print("pred:",torch.min(pred_mask_logits),torch.max(pred_mask_logits))
#     #print("gt:",torch.min(gt_masks),torch.max(gt_masks))
#     mask_loss_p = F.binary_cross_entropy_with_logits(pred_mask_logits, gt_masks, reduction="mean")

#     pred_mask_logits = -1*pred_mask_logits
#     gt_masks = (1-gt_masks)

#     mask_loss_n = F.binary_cross_entropy_with_logits(pred_mask_logits, gt_masks, reduction="mean")

#     return (mask_loss_p+mask_loss_n)/2



# detectron2.modeling.roi_heads.mask_head.mask_rcnn_loss.__code__ = mask_rcnn_loss.__code__


In [10]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=1) == 0  # Extra objects
    false_negatives = np.sum(matches, axis=0) == 0  # Missed objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
            cfg, mapper=MyMapper_copy_paste(cfg), sampler=sampler
        )

    def build_hooks(self):
      # copy of cfg
      cfg = self.cfg.clone()

      # build the original model hooks
      hooks = super().build_hooks()

      # add the best checkpointer hook
      hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD, 
                                        DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                                        "MaP IoU",
                                        "max",
                                        ))
      return hooks


    # weighted loss
    def run_step(self):
      self._trainer.iter = self.iter
      """
      Implement the standard training logic described above.
      """
      assert self._trainer.model.training, "[SimpleTrainer] model was changed to eval mode!"
      start = time.perf_counter()
      """
      If you want to do something with the data, you can wrap the dataloader.
      """
      data = next(self._trainer._data_loader_iter)
      if_pseudo, if_gt = [], []
      for d in data:
        if "livecell" in d["file_name"]:
          if_gt.append(d)
        else:
          if "astro" in d["file_name"] or "cort" in d["file_name"] or "shsy5y" in d["file_name"]:
            if_pseudo.append(d)
          else:
            if_gt.append(d)
      del data

      data_time = time.perf_counter() - start

      """
      If you want to do something with the losses, you can wrap the model.
      """
      losses_gt, losses_pseudo = torch.tensor(0), torch.tensor(0)
      # 分开训练
      if if_pseudo:
        loss_dict = self._trainer.model(if_pseudo)
        if isinstance(loss_dict, torch.Tensor):
            losses_pseudo = loss_dict
            loss_dict = {"total_loss": loss_dict}
        else:
            losses_pseudo = sum(loss_dict.values())

      if if_gt:
        loss_dict = self._trainer.model(if_gt)
        if isinstance(loss_dict, torch.Tensor):
            losses_gt = loss_dict
            loss_dict = {"total_loss": loss_dict}
        else:
            losses_gt = sum(loss_dict.values())

      if self.iter < self.cfg.PSEUDO_WEIGHT_RATIO[0]:
        losses = losses_gt
      elif self.cfg.PSEUDO_WEIGHT_RATIO[0] <= self.iter < self.cfg.PSEUDO_WEIGHT_RATIO[1]:
        alpha = self.cfg.PSEUDO_WEIGHT_MAX*(self.iter - self.cfg.PSEUDO_WEIGHT_RATIO[0])/\
        (self.cfg.PSEUDO_WEIGHT_RATIO[1] - self.cfg.PSEUDO_WEIGHT_RATIO[0])
        losses = losses_gt + losses_pseudo*alpha
      else:
        alpha = self.cfg.PSEUDO_WEIGHT_MAX
        losses = losses_gt + losses_pseudo*alpha

      """
      If you need to accumulate gradients or do something similar, you can
      wrap the optimizer with your custom `zero_grad()` method.
      """
      self._trainer.optimizer.zero_grad()
      if losses != torch.tensor(0):
        losses.backward()
        self._trainer._write_metrics(loss_dict, data_time)
        self._trainer.optimizer.step()

    # @classmethod
    # def build_optimizer(cls,cfg,model) -> torch.optim.Optimizer:
    #   """
    #   Build an optimizer from config.
    #   """
    #   params = get_default_optimizer_params(
    #       model,
    #       base_lr=cfg.SOLVER.BASE_LR,
    #       weight_decay_norm=cfg.SOLVER.WEIGHT_DECAY_NORM,
    #       bias_lr_factor=cfg.SOLVER.BIAS_LR_FACTOR,
    #       weight_decay_bias=cfg.SOLVER.WEIGHT_DECAY_BIAS,
    #   )
    #   return maybe_add_gradient_clipping(cfg, torch.optim.SGD)(
    #       params,
    #       lr=cfg.SOLVER.BASE_LR,
    #       momentum=cfg.SOLVER.MOMENTUM,
    #       nesterov=cfg.SOLVER.NESTEROV,
    #       weight_decay=cfg.SOLVER.WEIGHT_DECAY,
    #   )

    # @classmethod
    # def build_lr_scheduler(cls,cfg, optimizer) -> torch.optim.lr_scheduler._LRScheduler:
    #   """
    #   Build a LR scheduler from config.
    #   """
    #   name = cfg.SOLVER.LR_SCHEDULER_NAME

    #   if name == "WarmupMultiStepLR":
    #       steps = [x for x in cfg.SOLVER.STEPS if x <= cfg.SOLVER.MAX_ITER]
    #       if len(steps) != len(cfg.SOLVER.STEPS):
    #           logger = logging.getLogger(__name__)
    #           logger.warning(
    #               "SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. "
    #               "These values will be ignored."
    #           )
    #       sched = MultiStepParamScheduler(
    #           values=[cfg.SOLVER.GAMMA ** k for k in range(len(steps) + 1)],
    #           milestones=steps,
    #           num_updates=cfg.SOLVER.MAX_ITER,
    #       )
    #   elif name == "WarmupCosineLR":
    #       sched = CosineParamScheduler(1, 1/40)
    #   elif name == "CyclicLR":
    #       sched = torch.optim.lr_scheduler.CyclicLR(optimizer,
    #                                                 base_lr=cfg.SOLVER.BASE_LR, max_lr=5*cfg.SOLVER.BASE_LR,
    #                                                 base_momentum=0.85, max_momentum=0.95)

    #   else:
    #       raise ValueError("Unknown LR scheduler: {}".format(name))

    #   sched = WarmupParamScheduler(
    #       sched,
    #       cfg.SOLVER.WARMUP_FACTOR,
    #       min(cfg.SOLVER.WARMUP_ITERS / cfg.SOLVER.MAX_ITER, 1.0),
    #       cfg.SOLVER.WARMUP_METHOD,
    #   )
    #   return LRMultiplier(optimizer, multiplier=sched, max_iter=cfg.SOLVER.MAX_ITER)

In [11]:
def run():
  dataDir = "data/train"
  dataDir_livecell = "livecell-dataset/LIVECell_dataset_2021/images"
  DatasetCatalog.clear()
  MetadataCatalog.clear()
  register_coco_instances(f'sartorius_train',{}, 'data/LIVECell_dataset_2021/livecell_annotations_train.json'.format(fold), dataDir_livecell)
  register_coco_instances(f'sartorius_test',{}, 'data/LIVECell_dataset_2021/livecell_annotations_test.json'.format(fold), dataDir_livecell)
  register_coco_instances(f'sartorius_semi',{}, 'data/train_semi_1w.json'.format(fold), dataDir)
  #register_coco_instances(f'sartorius_val',{},'input/all/annotations_val.json'.format(fold), dataDir_livecell)

  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
  cfg.DATASETS.TRAIN = (f"sartorius_train","sartorius_test","sartorius_semi")
  #cfg.DATASETS.TEST = (f"sartorius_val",)
  # cfg.MODEL.PIXEL_MEAN = Config.pixel_mean
  # cfg.MODEL.PIXEL_STD = Config.pixel_std

  cfg.DATALOADER.NUM_WORKERS = 7
  #cfg.MODEL.WEIGHTS = "finetuned/livecell_cascade_v0/model_best.pth"#"/content/drive/MyDrive/Kaggle/Sartorius/model/pretrained/LIVECell_anchor_based_model.pth"
  cfg.SOLVER.IMS_PER_BATCH = 4
  #cfg.SOLVER.REFERENCE_WORLD_SIZE = 2

  cfg.SOLVER.BASE_LR = 5e-3
  cfg.SOLVER.WEIGHT_DECAY = 5e-5
  cfg.SOLVER.MAX_ITER = 20000#len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH * 20 #尝试20ep 用lr调整
  cfg.SOLVER.STEPS = (15000, 17500, 19000,)
  cfg.SOLVER.CHECKPOINT_PERIOD = 2000

  cfg.INPUT.MIN_SIZE_TRAIN = (440, 480, 520, 560, 580, 620, 660, 700, 760, 800)
  cfg.INPUT.MAX_SIZE_TRAIN = 1333
  cfg.INPUT.CROP.ENABLE = False
  cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
  cfg.INPUT.MIN_SIZE_TEST = 800
  cfg.INPUT.MAX_SIZE_TEST = 1333
  #cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16], [32], [64], [128], [256]] 
  cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10 
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
  cfg.MODEL.BACKBONE.FREEZE_AT = 0

  #cfg.TEST.EVAL_PERIOD = 
  cfg.TEST.EVAL_PERIOD = 0 #len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

  cfg.INPUT.MASK_FORMAT='bitmask'
  cfg.TEST.DETECTIONS_PER_IMAGE = 1000
  cfg.SOLVER.AMP.ENABLED  = False

  cfg.PSEUDO_WEIGHT_RATIO = [12000,18000,]
  cfg.PSEUDO_WEIGHT_MAX = 3
  cfg.SEED = 42
#   cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = Config.anchor_generators_aspect_ratios
#   cfg.MODEL.ANCHOR_GENERATOR.SIZES = Config.anchor_generators_sizes
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # #cfg.MODEL.FPN.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5","p6"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]

  # cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, True, True, True] # on Res3,Res4,Res5
  # cfg.MODEL.RESNETS.DEFORM_MODULATED = True
  # cfg.MODEL.RESNETS.DEFORM_NUM_GROUPS = 2
  # cfg.MODEL.RESNETS.NORM = "GN"
  # cfg.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
  # cfg.MODEL.ROI_BOX_HEAD.NUM_CONV = 4
  # cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 1
  # cfg.MODEL.ROI_BOX_HEAD.NORM = "GN"  
  # cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION = 7
  # cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True  
  # cfg.MODEL.ROI_MASK_HEAD.NUM_CONV = 8
  # cfg.MODEL.ROI_MASK_HEAD.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5]
  # cfg.MODEL.ROI_BOX_CASCADE_HEAD.IOUS = (0.5, 0.6, 0.7)

  cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 3000
  cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 3000
  
    
  # cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "ciou"
  # cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "ciou"
  # cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_WEIGHT = 10.0
  #cfg.SOLVER.AMP.ENABLED = True

  cfg.OUTPUT_DIR = "pretrained/final_weapon/".format(fold)

  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = Trainer(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

In [ ]:
# 2 的理论score最高
for fold in range(1,2):
  run()

# #run_all()

[12/27 19:40:38 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Some model parameters or buffers are not found in the checkpoint:
backbone.bottom_up.res3.0.conv2_offset.{bias, weight}
backbone.bottom_up.res3.1.conv2_offset.{bias, weight}
backbone.bottom_up.res3.2.conv2_offset.{bias, weight}
backbone.bottom_up.res3.3.conv2_offset.{bias, weight}
backbone.bottom_up.res3.4.conv2_offset.{bias, weight}
backbone.bottom_up.res3.5.conv2_offset.{bias, weight}
backbone.bottom_up.res3.6.conv2_offset.{bias, weight}
backbone.bottom_up.res3.7.conv2_offset.{bias, weight}
backbone.bottom_up.res4.0.conv2_offset.{bias, weight}
backbone.bottom_up.res4.1.conv2_offset.{bias, weight}
backbone.bottom_up.res4.10.conv2_offset.{bias, weight}
backbone.bottom_up.res4.11.conv2_offset.{bias, weight}
backbone.bottom_up.res4.12.conv2_offset.{bias, weight}
backbone.bottom_up.res4.13.conv2_offset.{bias, weight}
backbone.bottom_up.res4.14.conv2_offset.{bias, weight}
backbone.bottom_up.res4.15.conv2_offset.{bias, weight}
backbone.bottom_up.res4.16.conv2_offset.{bias, weight}
backbone.

[12/27 19:42:07 d2.engine.train_loop]: Starting training from iteration 0


[W LegacyTypeDispatch.h:74] Warning: AutoNonVariableTypeMode is deprecated and will be removed in 1.10 release. For kernel implementations please use AutoDispatchBelowADInplaceOrView instead, If you are looking for a user facing API to enable running your inference-only workload, please use c10::InferenceMode. Using AutoDispatchBelowADInplaceOrView in user code is under risk of producing silent wrong result in some edge cases. See Note [AutoDispatchBelowAutograd] for more details. (function operator())


[12/27 19:42:58 d2.utils.events]:  eta: 10:24:15  iter: 19  total_loss: 8.676  loss_cls_stage0: 2.202  loss_box_reg_stage0: 0.3103  loss_cls_stage1: 1.944  loss_box_reg_stage1: 0.1877  loss_cls_stage2: 2.014  loss_box_reg_stage2: 0.07774  loss_mask: 0.6932  loss_rpn_cls: 0.6953  loss_rpn_loc: 0.4994  time: 1.8300  data_time: 0.4651  lr: 9.9905e-05  max_mem: 27486M
[12/27 19:43:35 d2.utils.events]:  eta: 10:19:00  iter: 39  total_loss: 5.573  loss_cls_stage0: 1.179  loss_box_reg_stage0: 0.4963  loss_cls_stage1: 0.919  loss_box_reg_stage1: 0.298  loss_cls_stage2: 0.8041  loss_box_reg_stage2: 0.1016  loss_mask: 0.6928  loss_rpn_cls: 0.6431  loss_rpn_loc: 0.4249  time: 1.8441  data_time: 0.0763  lr: 0.0001998  max_mem: 29054M
[12/27 19:44:11 d2.utils.events]:  eta: 10:12:05  iter: 59  total_loss: 5.269  loss_cls_stage0: 1.087  loss_box_reg_stage0: 0.5017  loss_cls_stage1: 0.8493  loss_box_reg_stage1: 0.2978  loss_cls_stage2: 0.7404  loss_box_reg_stage2: 0.09524  loss_mask: 0.6921  loss_rpn